# SfM

In [1]:
import os

src_path = os.getcwd()
print(src_path)

/Users/unbinilium/Documents/Develop/Python/SfM/src


## Common Configs

In [2]:
datasets_path = '../data/datasets'
datasets_name = 'milkbox_dataset'

image_dir_path = os.path.join(src_path, datasets_path, datasets_name, 'images')
print(image_dir_path)

/Users/unbinilium/Documents/Develop/Python/SfM/src/../data/datasets/milkbox_dataset/images


## Generate Masks

In [3]:
import generate_masks_and_masked_images

Running inference on device 'cpu'


/Users/unbinilium/Documents/Develop/Python/SfM/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [4]:
image_masks_path = os.path.join(src_path, datasets_path, datasets_name, 'masks')
masked_image_path = os.path.join(src_path, datasets_path, datasets_name, 'masked_images')

In [5]:
generate_masks_and_masked_images.main(image_dir_path, image_masks_path, masked_image_path)

  0%|          | 0/43 [00:00<?, ?it/s]/Users/unbinilium/Documents/Develop/Python/SfM/lib/python3.10/site-packages/torch/nn/functional.py:3734: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/Users/unbinilium/Documents/Develop/Python/SfM/lib/python3.10/site-packages/torch/nn/functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
100%|██████████| 43/43 [01:43<00:00,  2.40s/it]


## Extract Features

In [6]:
import extract_features

In [7]:
superpoint_conf = {
    'output': 'feats-spp',
    'model': {
        'path': 'thirdparty/SuperPointPretrainedNetwork/superpoint_v1.pth',
        'name': 'spp_det'
    },
    'preprocessing': {
        'grayscale': True
    },
    'conf': {
        'descriptor_dim': 256,
        'nms_radius': 3,
        'max_keypoints': 4096,
        'keypoints_threshold': 0.6
    }
}

features_out_path = os.path.join(src_path, datasets_path, datasets_name, superpoint_conf['output'] + '.h5')
print(features_out_path)

/Users/unbinilium/Documents/Develop/Python/SfM/src/../data/datasets/milkbox_dataset/feats-spp.h5


In [8]:
extract_features.main(image_masks_path, features_out_path, superpoint_conf)

=> load weights:  thirdparty/SuperPointPretrainedNetwork/superpoint_v1.pth


  0%|          | 0/43 [00:00<?, ?it/s]/Users/unbinilium/Documents/Develop/Python/SfM/lib/python3.10/site-packages/torch/nn/functional.py:4227: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
100%|██████████| 43/43 [00:59<00:00,  1.39s/it]


## Generate Pairs

In [9]:
import generate_pairs

In [10]:
generate_pairs_conf = {
    'method': 'sequential'
}

pairs_out_path = os.path.join(
    src_path,
    datasets_path,
    datasets_name,
    '{}-pairs.txt'.format(generate_pairs_conf['method']))
print(pairs_out_path)

/Users/unbinilium/Documents/Develop/Python/SfM/src/../data/datasets/milkbox_dataset/sequential-pairs.txt


In [11]:
generate_pairs.main(image_dir_path, pairs_out_path, generate_pairs_conf)

## Matching Features

In [12]:
import match_features

In [13]:
match_features_conf = {
    'model': {
        'path': 'thirdparty/SuperGluePretrainedNetwork/models/weights/superglue_indoor.pth'
    },
    'output': 'matches-spg',
    'conf': {
        'descriptor_dim': 256,
        'weights': 'indoor',
        'match_threshold': 0.7
    }
}

matches_out_path = os.path.join(
    src_path,
    datasets_path,
    datasets_name,
    '{}.h5'.format(match_features_conf['output']))
print(matches_out_path)

/Users/unbinilium/Documents/Develop/Python/SfM/src/../data/datasets/milkbox_dataset/matches-spg.h5


In [14]:
match_features.main(features_out_path, pairs_out_path, matches_out_path, match_features_conf)

Loaded SuperGlue model ("indoor" weights)
=> load weights:  thirdparty/SuperGluePretrainedNetwork/models/weights/superglue_indoor.pth


100%|██████████| 43/43 [00:01<00:00, 35.06it/s]
